# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [8]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

s = 0
for sqrt_value in generator:
    s += sqrt_value
print(f"sum of squares with limit = 5 is: {s}")

g = square_root_generator(13)
i = 1
while i < 13:
    next(g)
    i += 1
print(f"square root of 13 is: {next(g)}")

sum of squares with limit = 5 is: 8.382332347441762
square root of 13 is: 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [19]:
import dlt
import duckdb

persons_pipeline = dlt.pipeline(destination='duckdb', dataset_name='persons')

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

info = persons_pipeline.run(people_1(), table_name="people", write_disposition="replace")

print(info)

conn = duckdb.connect(f"{persons_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{persons_pipeline.dataset_name}'")
x = conn.sql("select sum(Age) from people").df()
print(x)
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

info = persons_pipeline.run(people_2(), table_name="people", write_disposition="append")

print(info)

x = conn.sql("select sum(Age) from people").df()
print(x)

Pipeline dlt_ipykernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons
The duckdb destination used duckdb:////home/m13salama/csed/Data-Zoom-Camp/workshop1/workshops/dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708272307.757592 is LOADED and contains no failed jobs
   sum(Age)
0     140.0
Pipeline dlt_ipykernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons
The duckdb destination used duckdb:////home/m13salama/csed/Data-Zoom-Camp/workshop1/workshops/dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708272308.4719362 is LOADED and contains no failed jobs
   sum(Age)
0     353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [20]:
persons_pipeline = dlt.pipeline(destination='duckdb', dataset_name='persons_merge')
info = persons_pipeline.run(people_1(), table_name="people",primary_key="ID", write_disposition="replace")
print(info)

conn = duckdb.connect(f"{persons_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{persons_pipeline.dataset_name}'")
x = conn.sql("select sum(Age) from people").df()
print(x)

info = persons_pipeline.run(people_2(), table_name="people",primary_key="ID", write_disposition="merge")
print(info)

x = conn.sql("select sum(Age) from people").df()
print(x)

Pipeline dlt_ipykernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_merge
The duckdb destination used duckdb:////home/m13salama/csed/Data-Zoom-Camp/workshop1/workshops/dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708273737.611089 is LOADED and contains no failed jobs
   sum(Age)
0     140.0
Pipeline dlt_ipykernel_launcher load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_merge
The duckdb destination used duckdb:////home/m13salama/csed/Data-Zoom-Camp/workshop1/workshops/dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708273738.2834673 is LOADED and contains no failed jobs
   sum(Age)
0     266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX